In [1]:
#pylint: disable=import-error, wrong-import-position
import os
import sys
import json

import numpy as np
from sklearn import metrics

from qiskit import BasicAer
from qiskit.visualization import circuit_drawer
from qiskit.algorithms.optimizers import SPSA
from qiskit.providers.ibmq.runtime.utils import RuntimeEncoder, RuntimeDecoder
from qiskit.providers.ibmq.runtime import UserMessenger

# Put qiskit-machine-learning subtree at front of Python path
ml_path = os.path.abspath(os.path.join('../../qiskit-machine-learning'))
sys.path.insert(0, ml_path)
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.utils.loss_functions import SVCLoss
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.runtime import QuantumKernelTrainerClient

# Put this repository on the Python path and import qkt pkgs
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path)
from qkt.feature_maps import CovariantFeatureMap
from qkt.utils import train_test_split

# Load 7 qubit dataset

In [2]:
# Load the dataset and split into train and test sets
DATA_FILEPATH = '../../data/dataset_graph7.csv'
X_train, y_train, X_test, y_test = train_test_split(DATA_FILEPATH)
data_list = X_train.tolist()
labels_list = y_train.tolist()

# Set up feature map

In [3]:
num_features = np.shape(X_train)[1]
entangler_map = [[0,2],[3,4],[2,5],[1,4],[2,3],[4,6]]

# Note that [[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]]
# is a suitable input for the 10-qubit dataset

fm = CovariantFeatureMap(
    feature_dimension=num_features,
    entanglement=entangler_map,
    single_training_parameter=True
)

circuit_drawer(fm)

┌──────────────┐             ░ ┌─────────────────┐ ┌─────────────────┐ 
q_0: ┤ Ry(θ_par[0]) ├─■───────────░─┤ Rz(-2*x_par[1]) ├─┤ Rx(-2*x_par[0]) ├─
     ├──────────────┤ │           ░ ├─────────────────┤ ├─────────────────┤ 
q_1: ┤ Ry(θ_par[0]) ├─┼─────■─────░─┤ Rz(-2*x_par[3]) ├─┤ Rx(-2*x_par[2]) ├─
     ├──────────────┤ │     │     ░ ├─────────────────┤ ├─────────────────┤ 
q_2: ┤ Ry(θ_par[0]) ├─■──■──┼──■──░─┤ Rz(-2*x_par[5]) ├─┤ Rx(-2*x_par[4]) ├─
     ├──────────────┤    │  │  │  ░ ├─────────────────┤ ├─────────────────┤ 
q_3: ┤ Ry(θ_par[0]) ├─■──┼──┼──■──░─┤ Rz(-2*x_par[7]) ├─┤ Rx(-2*x_par[6]) ├─
     ├──────────────┤ │  │  │     ░ ├─────────────────┤ ├─────────────────┤ 
q_4: ┤ Ry(θ_par[0]) ├─■──┼──■──■──░─┤ Rz(-2*x_par[9]) ├─┤ Rx(-2*x_par[8]) ├─
     ├──────────────┤    │     │  ░ ├─────────────────┴┐├─────────────────┴┐
q_5: ┤ Ry(θ_par[0]) ├────■─────┼──░─┤ Rz(-2*x_par[11]) ├┤ Rx(-2*x_par[10]) ├
     ├──────────────┤          │  ░ ├──────────────────┤├──────────────────┤
q_6: ┤ Ry(θ_par[0]) ├──────────■──░─┤ Rz(-2*x_par[13]) ├┤ Rx(-2*x_par[12]) ├
     └──────────────┘             ░ └──────────────────┘└──────────────────┘

# Set up the QKTRuntimeClient object

In [4]:
SHOTS = 1024
MEASUREMENT_ERROR_MITIGATION = False

# Use QASM simulator backend
backend = BasicAer.get_backend('qasm_simulator')

# Instantiate quantum kernel
quant_kernel = QuantumKernel(fm,
                             user_parameters=fm.user_parameters,
                             quantum_instance=backend)

# Set up the optimizer
optimizer = SPSA(maxiter=10,
                learning_rate=0.1,
                perturbation=0.1
            )

# Set up the loss callable
loss_func = SVCLoss().get_variational_callable(
                quant_kernel, X_train, y_train
)

inputs = {
    "quantum_kernel": quant_kernel,
    "data": X_train,
    "labels": y_train,
    "optimizer": optimizer,
    "shots": SHOTS,
    "measurement_error_mitigation": MEASUREMENT_ERROR_MITIGATION,
}

serialized_inputs = json.dumps(inputs, cls=RuntimeEncoder)
deserialized_inputs = json.loads(serialized_inputs, cls=RuntimeDecoder)
qk = deserialized_inputs['quantum_kernel']
up = qk.user_parameters
print(qk.user_param_binds)

#deserialized_inputs.quantum_kernel.feature_map.draw()
# Instantiate the QKTRuntimeClient object
#qkt_program = QuantumKernelTrainerClient(
#    quantum_kernel=quant_kernel,
#    backend=backend,
#    loss=loss_func,
#    optimizer=optimizer,
#    shots=SHOTS,
#    measurement_error_mitigation=MEASUREMENT_ERROR_MITIGATION
#)

{ParameterExpression(θ_par[0]): ParameterExpression(θ_par[0])}


# Fit the kernel via runtime and retrieve the optimized quantum kernel

In [7]:
#qkt_results = qkt_program.fit_kernel(X_train, y_train)
#optimized_kernel = qkt_results.quantum_kernel

# Analyze performance on test data using an SVC model

In [8]:
# Use QSVC to test the kernel
#qsvc = QSVC(quantum_kernel=optimized_kernel)

# Fit the QSVC
#qsvc.fit(X_train, y_train)

# Predict the labels
#labels_test = qsvc.predict(X_test)

# Measure unweighted accuracy
#accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
#print(f"accuracy test: {accuracy_test}")